## 1. タイトル列のユニーク値を抽出しCSV保存

### 1.1 ライブラリのインポート

In [1]:
import time
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### 1.2 ファイルパスの設定

In [2]:
# ▼ 入力CSVファイルのパス
input_csv_path = "gamei_apps_sales_summary_utf8bom_ver2.csv"

# ▼ 出力CSVファイルのパス
output_csv_path = "unique_titles.csv"

### 1.3 CSVの読み込み

In [3]:
# ▼ CSVの読み込み
df = pd.read_csv(input_csv_path)

# ▼ 読み込み確認（先頭5行）
df.head()

,タイトル,年月,売上予測,関連ジャンル,サービス終了日
0,スーパーガンダムロワイヤル,2018-12,2.48億G,NaN,2021-07-29
1,スーパーガンダムロワイヤル,2018-11,2.51億G,NaN,2021-07-29
2,スーパーガンダムロワイヤル,2018-10,2.63億G,NaN,2021-07-29
3,スーパーガンダムロワイヤル,2018-09,1.90億G,NaN,2021-07-29
4,スーパーガンダムロワイヤル,2018-08,1.98億G,NaN,2021-07-29


### 1.4 タイトル列のユニーク化

In [4]:
# ▼ タイトル列の重複を除いた一覧を取得
unique_titles = df["タイトル"].dropna().unique()

# ▼ 結果の確認
print(f"ユニークなタイトル数: {len(unique_titles)}")
print(unique_titles[:10])  # 最初の10件だけ確認

ユニークなタイトル数: 393
['スーパーガンダムロワイヤル' '武器よさらば' 'スーパーロボット大戦X-Ω' 'FINAL FANTASY GRANDMASTERS'
 'ドラゴンプロジェクト' '戦の海賊' '誰ガ為のアルケミスト' 'デッキヒーローズ-本格派戦略カードゲーム-'
 '美少女戦士セーラームーン\u3000セーラームーンドロップス' 'モンスターハンター エクスプロア']


### 1.5 データフレームに変換して保存

In [5]:
# ▼ データフレームに変換
titles_df = pd.DataFrame(unique_titles, columns=["タイトル"])

# ▼ 新しいCSVとして保存（UTF-8 BOM付きで文字化け防止）
titles_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

print(f"{len(titles_df)} 件のユニークなタイトルを {output_csv_path} に保存しました。")

393 件のユニークなタイトルを unique_titles.csv に保存しました。


## 2 Seleniumで4Gamerからゲーム発売日を自動取得

In [6]:
# ▼ ファイル設定
input_file = "unique_titles.csv"
output_file = "titles_with_release_dates_test.csv"

# ▼ Chromeオプション（必要に応じて headless）
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")

# ▼ ドライバ起動
driver = webdriver.Chrome(options=options)

### 2.1 ライブラリ・ドライバの準備

In [7]:
# ▼ ファイル設定
input_file = "unique_titles.csv"
output_file = "apps_release_dates.csv"

# ▼ Chromeオプション（必要に応じて headless）
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")

# ▼ ドライバ起動
driver = webdriver.Chrome(options=options)

### 2.2 タイトル一覧の読み込みと準備

In [8]:
# ▼ タイトル読み込み
df = pd.read_csv(input_file).copy()
df["発売日"] = None
df

,タイトル,発売日
0,スーパーガンダムロワイヤル,None
1,武器よさらば,None
2,スーパーロボット大戦X-Ω,None
3,FINAL FANTASY GRANDMASTERS,None
4,ドラゴンプロジェクト,None
...,...,...
388,エレメンタルストーリーワールド,None
389,モルガナティック・アイドル 乙女ゲーム・恋愛ゲーム,None
390,廻らぬ星のステラリウム,None
391,タップハンター～剣と魔法の放置RPG～,None


In [11]:
# ▼ メイン処理ループの開始
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{len(df)}] 検索中：{title}")

    try:
        # ① トップページから検索
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # ② 検索結果リンクを待つ
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        )
        links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]

        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            continue

        # ③ ゲーム紹介ページで発売日を取得
        release_date = None
        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")
                for th in th_list:
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            release_date = td.text.strip()
                            break
                        except:
                            continue

                if release_date:
                    df.at[idx, "発売日"] = release_date
                    print(f" → 発売日：{release_date}")
                    break

            except Exception as sub_e:
                print(" → ページ処理中エラー:")
                traceback.print_exc()
                continue

        if not release_date:
            print(" → 発売日が見つかりません")

    except Exception as e:
        print(" → メイン処理エラー:")
        traceback.print_exc()
        continue

[1/393] 検索中：スーパーガンダムロワイヤル
 → 発売日：2015/10/16
[2/393] 検索中：武器よさらば
 → 発売日：2017/03/30
[3/393] 検索中：スーパーロボット大戦X-Ω
 → 発売日：2015/10/05
[4/393] 検索中：FINAL FANTASY GRANDMASTERS
 → 発売日：2015/10/01
[5/393] 検索中：ドラゴンプロジェクト
 → 発売日：2016/06/03
[6/393] 検索中：戦の海賊
 → 発売日：2015/08/27
[7/393] 検索中：誰ガ為のアルケミスト
 → 発売日：2016/01/28
[8/393] 検索中：デッキヒーローズ-本格派戦略カードゲーム-
 → メイン処理エラー:
[9/393] 検索中：美少女戦士セーラームーン　セーラームーンドロップス


Traceback (most recent call last):
  File "C:\Users\zakky\AppData\Local\Temp\ipykernel_4664\3206948621.py", line 17, in <module>
    WebDriverWait(driver, 10).until(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



 → 発売日：2015/09/03
[10/393] 検索中：モンスターハンター エクスプロア
 → 発売日：2015/09/29
[11/393] 検索中：夢色キャスト
 → 発売日：2024/09/29
[12/393] 検索中：グリムノーツ
 → 発売日：2016/01/21
[13/393] 検索中：DAME×PRINCE -ダメ王子たちとのドタバタ恋愛ADV
 → 発売日：2016/03/31
[14/393] 検索中：かんぱにガールズ ファンタジーRPG
 → メイン処理エラー:
[15/393] 検索中：デジモンリンクス


Traceback (most recent call last):
  File "C:\Users\zakky\AppData\Local\Temp\ipykernel_4664\3206948621.py", line 17, in <module>
    WebDriverWait(driver, 10).until(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



 → 発売日：2016/03/24
[16/393] 検索中：BFBチャンピオンズ2.0
 → 発売日：2016/06/29
[17/393] 検索中：フェアリーテイル 極・魔法乱舞
 → 発売日：2015/12/04
[18/393] 検索中：12オーディンズ - 王道RPG
 → メイン処理エラー:
[19/393] 検索中：ラストピリオド - 終わりなき螺旋の物語 -


...

Traceback (most recent call last):
  File "C:\Users\zakky\AppData\Local\Temp\ipykernel_4664\3206948621.py", line 17, in <module>
    WebDriverWait(driver, 10).until(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



 → 発売日：2018/07/26
[45/393] 検索中：ソードアート・オンライン メモリー・デフラグ
 → 発売日：2016/08/29
[46/393] 検索中：バンドやろうぜ！
 → 発売日：2016/10/12
[47/393] 検索中：銀魂 かぶき町大活劇
 → 発売日：2016/09/15
[48/393] 検索中：ソウルリバース ゼロ（SOUL REVERSE ZERO）
 → メイン処理エラー:
[49/393] 検索中：モバサカ CHAMPIONS MANAGER


Traceback (most recent call last):
  File "C:\Users\zakky\AppData\Local\Temp\ipykernel_4664\3206948621.py", line 17, in <module>
    WebDriverWait(driver, 10).until(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\zakky\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



 → 発売日：-
[50/393] 検索中：恋愛幕末カレシ～恋愛ゲーム・乙女ゲーム女性向け～


KeyboardInterrupt: 

In [ ]:
# ▼ ドライバ終了＆CSV保存
driver.quit()
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n 処理完了 → {output_file} に保存されました。")